This notebook creates a tensorflow `tf.data.Dataset` object from images on disk

In [1]:
import os
import tensorflow as tf
tf.enable_eager_execution()
import pandas as pd
import numpy as np
import sklearn.model_selection
import sklearn.preprocessing

In [2]:
# Parameters
image_dims = (28, 28, 1)
batchsize = 64
num_labels = 10

In [3]:
data_training, data_validation = sklearn.model_selection.train_test_split(
    pd.read_csv("train.csv"), test_size=0.2)
data_test = pd.read_csv("test.csv")

X_training = data_training.drop("label", axis=1).values
X_validation = data_validation.drop("label", axis=1).values
X_test = data_test.values.reshape((-1,) + image_dims)

y_training = data_training["label"]
y_validation = data_validation["label"]

In [5]:
def preprocess_training_data(image, label):
    """
    Converts 1D array into 2D matrix and normalized entries
    """
    image = tf.reshape(image, image_dims)
    image = tf.dtypes.cast(image, tf.float64)
    min_val = tf.reduce_min(image)
    max_val = tf.reduce_max(image)
    image = (image - min_val) / (max_val - min_val)
    label = tf.one_hot(indices=label, depth=num_labels)
    return image, label

def preprocess_testing_data(image):
    """
    Converts 1D array into 2D matrix and normalized entries
    """
    image = tf.reshape(image, image_dims)
    image = tf.dtypes.cast(image, tf.float64)
    min_val = tf.reduce_min(image)
    max_val = tf.reduce_max(image)
    image = (image - min_val) / (max_val - min_val)
    return image

In [6]:
def make_dataset(data, labels=None):
    if labels is None:
        dataset = tf.data.Dataset.from_tensor_slices(data)
        dataset = dataset.map(preprocess_testing_data, num_parallel_calls=-1)
    else:
        dataset = tf.data.Dataset.from_tensor_slices((data, labels))
        dataset = dataset.shuffle(buffer_size=len(data))
        dataset = dataset.map(preprocess_training_data, num_parallel_calls=-1)
    dataset = dataset.batch(batchsize)
    dataset = dataset.prefetch(1)
    return dataset

In [7]:
dataset_training = make_dataset(X_training, y_training)
dataset_validation = make_dataset(X_validation, y_validation)
dataset_testing = make_dataset(X_test)

In [8]:
def make_model():
    input_layer = tf.keras.layers.Input(
        shape=image_dims, name="Input")
    layer = tf.keras.layers.Conv2D(
        filters=64, kernel_size=(3, 3), 
        activation="relu", use_bias=True,
        kernel_initializer="glorot_uniform", 
        bias_initializer="glorot_uniform")(input_layer)
    layer = tf.keras.layers.Dropout(rate=0.3)(layer)
    layer = tf.keras.layers.Conv2D(
        filters=64, kernel_size=(3, 3), 
        activation="relu", use_bias=True,
        kernel_initializer="glorot_uniform", 
        bias_initializer="glorot_uniform")(layer)
    layer = tf.keras.layers.Dropout(rate=0.3)(layer)
    layer = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(layer)
    layer = tf.keras.layers.Conv2D(
        filters=128, kernel_size=(3, 3), 
        activation="relu", use_bias=True,
        kernel_initializer="glorot_uniform", 
        bias_initializer="glorot_uniform")(layer)
    layer = tf.keras.layers.Dropout(rate=0.3)(layer)
    layer = tf.keras.layers.Conv2D(
        filters=128, kernel_size=(3, 3), 
        activation="relu", use_bias=True,
        kernel_initializer="glorot_uniform", 
        bias_initializer="glorot_uniform")(layer)
    layer = tf.keras.layers.Dropout(rate=0.3)(layer)
    layer = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(layer)
    layer = tf.keras.layers.Flatten()(layer)
    output_layer = tf.keras.layers.Dense(units=num_labels, activation="softmax")(layer)
    model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer="adam", 
        loss="categorical_crossentropy",
        metrics=["accuracy"])
    return model

In [9]:
model = make_model()

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
dropout (Dropout)            (None, 26, 26, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        36928     
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 128)       73856 

In [11]:
model.fit(dataset_training, epochs=5, validation_data=dataset_validation)

Epoch 1/5


W0703 23:49:49.990462 140453180405504 deprecation.py:323] From /home/jan/anaconda3/envs/py3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


525/525 [==============================] - 91s 174ms/step - loss: 0.2682 - acc: 0.9144 - val_loss: 0.1026 - val_acc: 0.9757
Epoch 2/5
525/525 [==============================] - 87s 166ms/step - loss: 0.0745 - acc: 0.9772 - val_loss: 0.0725 - val_acc: 0.9819
Epoch 3/5
525/525 [==============================] - 85s 161ms/step - loss: 0.0532 - acc: 0.9831 - val_loss: 0.0559 - val_acc: 0.9870
Epoch 4/5
525/525 [==============================] - 86s 163ms/step - loss: 0.0429 - acc: 0.9867 - val_loss: 0.0410 - val_acc: 0.9890
Epoch 5/5
525/525 [==============================] - 87s 165ms/step - loss: 0.0355 - acc: 0.9889 - val_loss: 0.0477 - val_acc: 0.9888


In [12]:
pred = model.predict(dataset_testing)

In [29]:
output = pd.DataFrame(
    data={
        "ImageId": data_test.index+1, 
        "Label": np.argmax(pred, axis=1)})

output.head()
output.to_csv("TestPrediction.csv", index=False)